In [ ]:
# import maths
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from np.random import randint  

import os
import csv

from glob import glob
import matplotlib.pyplot as plt
from collections import namedtuple
from copy import deepcopy
from tqdm import tqdm
import random

import cv2

# pytorch + torchvision
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter 
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.io as io

import kornia

# Check that you're using a recent OpenCV version.
assert cv2.__version__ > '4.5', 'Please use OpenCV 4.5 or later.'

Write down the plan in words and references. We will then fill in code.

In [ ]:
# Encoding: ResNeXt (conv 1-3) => Position Enc. (ref. 2107.14222) => Deep-ViT (w/ EfficientNet)
# Decoding(for unsupervised training) : HiT(low resolution stage, same # as Deep-ViT) => FCC-GAN
# Comparison : Transformer-XL 
# Classifier/MLP : MLP head (output 8) => Reconstruction module => F

# Training Steps:
# 1. Train Encoder (unsupervised): manipulate input image (ref. SiT, + rotation) and match to output
# 2. Train Comparison (unsupervised) : use different head, mix&match the 2 images (ref. BERT, ALBERT)
# 3. Train MLP (supervised) : compare output to F

# Inference Steps: Encode each image => concat. 2 images => Comparison => MLP

#offical sample code: https://www.kaggle.com/code/eduardtrulls/imc2022-training-data?scriptVersionId=92062607

# Load data

In [ ]:
# copied from sample code
# Input data files are available in the read-only "../input/" directory.

# on kaggle
src = '../input/image-matching-challenge-2022/train'

# on pc
# src = './image-matching-challenge-2022/train'

val_scenes = []
for f in os.scandir(src):
    if f.is_dir():
        cur_scene = os.path.split(f)[-1]
        print(f'Found scene "{cur_scene}"" at {f.path}')
        val_scenes += [cur_scene]

In [ ]:
# Each scene in the validation set contains a list of images, poses, and pairs. Let's pick one and look at some images.

scene = 'piazza_san_marco'

images_dict = {}
for filename in glob(f'{src}/{scene}/images/*.jpg'):
    cur_id = os.path.basename(os.path.splitext(filename)[0])

    # OpenCV expects BGR, but the images are encoded in standard RGB, so you need to do color conversion if you use OpenCV for I/O.
    images_dict[cur_id] = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
    
print(f'Loaded {len(images_dict)} images.')

num_rows = 6
num_cols = 4
f, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20), constrained_layout=True)
for i, key in enumerate(images_dict):
    if i >= num_rows * num_cols:
        break
    cur_ax = axes[i % num_rows, i // num_rows]
    cur_ax.imshow(images_dict[key])
    cur_ax.set_title(key)
    cur_ax.axis('off')

In [ ]:
# dataset

# this loads 2 imgs at once for the first 2 training steps
class IMC_images(Dataset):

    def __init__(self, landmarks, imgs_file, root_dir, transform=None):
        self.landmarks = landmarks
        self.root_dir = root_dir
        self.transform = transform
        self.buffer_data()

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        landmark1 = random.randint(0, len(self.landmarks))
        landmark2 = random.randint(0, len(self.landmarks))

        img_num1 = random.randint(0, len(self.images[landmark1]))
        img_num1 = random.randint(0, len(self.images[landmark2]))

        img1 = self.imgs[landmark1][img_num1]
        img2 = self.imgs[landmark2][img_num2]

        comp = []
        if landmark1 == landmark2 :
            comp = [1, 0]
        else 
            comp = [0, 1]
        
        

        sample = {'img1': img1, 'img2': img2, 'comp': comp}

        return sample
    
    def buffer_data(self):
        self.imgs = {}
        for i, j in enumerate(self.landmarks):
            self.imgs[i] = self.transform(
                torch_vision.decode_image(file) for file in glob.glob(self.root_dir + j + '/*.jpeg')
            ).cpu() # Length * Height * Width


# dataset
class IMC_dataset(Dataset):
    """Image Matching Challenge 2022 dataset"""

    def __init__(self, cali_file, covis_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.calibration = pd.read_csv(cali_file)
        self.pair_covisibility = pd.read_csv(covis_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.pair_covisibility)

    def __getitem__(self, idx):
        """
        Returns:
            image_1 (tensor): First image
            image_2 (tensor): Secound image
            covisibility (float)
            K_1 (matrix): Camera intransic matrix
            K_2 (matrix): Camera intransic matrix
            R_1 (matrix): Rotation matrix
            R_2 (matrix): Rotation matrix
            T_1 (vector): Translation vector
            T_2 (vector): Translation vector
            F (matrix): Fundamental matrix
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        pairID = self.pair_covisibility["pair"][idx]

        img_1_id , img_2_id = pairID.split("-")

        F = self.pair_covisibility["fundamental_matrix"][idx]

        print(img_1_id, img_2_id) 

        img_1 = io.read_image(os.path.join(root_dir, img_1_id) + ".jpg")
        img_2 = io.read_image(os.path.join(root_dir, img_2_id) + ".jpg")

        covis = self.pair_covisibility["covisibility"][idx]

        idx_1 = self.calibration.loc[self.calibration['image_id'] == img_1_id]
        idx_2 = self.calibration.loc[self.calibration['image_id'] == img_2_id]

        K_1 = self.calibration['camera_intrinsics'][idx_1]
        K_2 = self.calibration['camera_intrinsics'][idx_2]

        R_1 = self.calibration['rotation_matrix'][idx_1]
        R_2 = self.calibration['rotation_matrix'][idx_2]

        T_1 = self.calibration['translation_vector'][idx_1]
        T_2 = self.calibration['translation_vector'][idx_2]

        sample = {'img_1': img_1, 'img_2': img_2, 'img_2': img_2, 'img_2': img_2, \
         'covisibility': covis, 'K_1': K_1, 'K_2': K_2, 'R_1': R_1, 'R_2': R_2, \
         'T_1': T_1, 'T_2': T_2, "F": F}

        if self.transform:
            sample = self.transform(sample)

        return sample
        
cali_file = '.\\image-matching-challenge-2022\\train\\brandenburg_gate\\calibration.csv'
covis_file = ".\\image-matching-challenge-2022\\train\\brandenburg_gate\\pair_covisibility.csv"
root_dir = ".\\image-matching-challenge-2022\\train\\brandenburg_gate\\images"

dataset = IMC_dataset(cali_file, covis_file, root_dir)

In [ ]:
# pytorch data loading
def get_scene_trainloader(scene):

    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = "./image-matching-challenge-2022/train/" + scene

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                            shuffle=True, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))


    # get some random training images
    dataiter = iter(trainloader)

    print(dataiter.next())
    images, labels = dataiter.next()

    # show images
    imshow(torchvision.utils.make_grid(images))
    # print labels
    print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

    return trainloader

get_scene_trainloader("piazza_san_marco")

In [ ]:
# image manipulation (ref. SiT)
# paper:
# github: github.com/Sara-Ahmed/SiT
# Copy + Pasted from the github 

def drop_rand_patches(X, X_rep=None, max_drop=0.3, max_block_sz=0.25, tolr=0.05):
    #######################
    # X_rep: replace X with patches from X_rep. If X_rep is None, replace the patches with Noise
    # max_drop: percentage of image to be dropped
    # max_block_sz: percentage of the maximum block to be dropped
    # tolr: minimum size of the block in terms of percentage of the image size
    #######################
    
    C, H, W = X.size()
    n_drop_pix = np.random.uniform(0, max_drop)*H*W
    mx_blk_height = int(H*max_block_sz)
    mx_blk_width = int(W*max_block_sz)
    
    tolr = (int(tolr*H), int(tolr*W))
    
    total_pix = 0
    while total_pix < n_drop_pix:
        
        # get a random block by selecting a random row, column, width, height
        rnd_r = randint(0, H-tolr[0])
        rnd_c = randint(0, W-tolr[1])
        rnd_h = min(randint(tolr[0], mx_blk_height)+rnd_r, H) #rnd_r is alread added - this is not height anymore
        rnd_w = min(randint(tolr[1], mx_blk_width)+rnd_c, W)
        
        if X_rep is None:
            X[:, rnd_r:rnd_h, rnd_c:rnd_w] = torch.empty((C, rnd_h-rnd_r, rnd_w-rnd_c), dtype=X.dtype, device='cuda').normal_()
        else:
            X[:, rnd_r:rnd_h, rnd_c:rnd_w] = X_rep[:, rnd_r:rnd_h, rnd_c:rnd_w]    
         
        total_pix = total_pix + (rnd_h-rnd_r)*(rnd_w-rnd_c)

    return X

def rgb2gray_patch(X, tolr=0.05):

    C, H, W = X.size()
    tolr = (int(tolr*H), int(tolr*W))
     
    # get a random block by selecting a random row, column, width, height
    rnd_r = randint(0, H-tolr[0])
    rnd_c = randint(0, W-tolr[1])
    rnd_h = min(randint(tolr[0], H)+rnd_r, H) #rnd_r is alread added - this is not height anymore
    rnd_w = min(randint(tolr[1], W)+rnd_c, W)
    
    X[:, rnd_r:rnd_h, rnd_c:rnd_w] = torch.mean(X[:, rnd_r:rnd_h, rnd_c:rnd_w], dim=0).unsqueeze(0).repeat(C, 1, 1)

    return X
    

def smooth_patch(X, max_kernSz=15, gauss=5, tolr=0.05):

    #get a random kernel size (odd number)
    kernSz = 2*(randint(3, max_kernSz+1)//2)+1
    gausFct = np.random.rand()*gauss + 0.1 # generate a real number between 0.1 and gauss+0.1
    
    C, H, W = X.size()
    tolr = (int(tolr*H), int(tolr*W))
     
    # get a random block by selecting a random row, column, width, height
    rnd_r = randint(0, H-tolr[0])
    rnd_c = randint(0, W-tolr[1])
    rnd_h = min(randint(tolr[0], H)+rnd_r, H) #rnd_r is alread added - this is not height anymore
    rnd_w = min(randint(tolr[1], W)+rnd_c, W)
    
    
    gauss = kornia.filters.GaussianBlur2d((kernSz, kernSz), (gausFct, gausFct))
    X[:, rnd_r:rnd_h, rnd_c:rnd_w] = gauss(X[:, rnd_r:rnd_h, rnd_c:rnd_w].unsqueeze(0))
    
    return X

def random_rotation(X, orientation=None):
    # generate random number between 0 and n_rot to represent the rotation
    if orientation is None:
        orientation = np.random.randint(0, 4)
    
    if orientation == 0: # do nothing
        pass
    elif orientation == 1:  
        X = X.rot90(-1, [1, 2])
    elif orientation == 2:  
        X = X.rot90(-1, [1, 2]).rot90(-1, [1, 2])
    elif orientation == 3: 
        X = X.rot90(1, [1, 2])
        
    return X, orientation

def distortImages(samples):
    n_imgs = samples.size()[0] #this is batch size, but in case bad inistance happened while loading
    samples_aug = samples.detach().clone()
    for i in range(n_imgs):
        samples_aug[i] = random_rotation(samples_aug[i])
        
        samples_aug[i] = rgb2gray_patch(samples_aug[i])

        samples_aug[i] = smooth_patch(samples_aug[i])

        samples_aug[i] = drop_rand_patches(samples_aug[i])

        idx_rnd = randint(0, n_imgs)
        if idx_rnd != i:
            samples_aug[i] = drop_rand_patches(samples_aug[i], samples_aug[idx_rnd])
      
    return samples_aug   

In [ ]:
# loss 
from pytorch_wavelets import DWTInverse, DWTForward

class DistillerLoss(nn.Module):
    def __init__(
            self,
            loss_weights={"l1": 1.0, "l2": 1.0, "loss_p": 1.0, "loss_g": 0.5}
    ):
        super().__init__()
        # l1/l2 loss
        self.l1_loss = nn.L1Loss()
        self.l2_loss = nn.MSELoss()
        
        # loss weights
        self.loss_weights = loss_weights
        # utils
        self.dwt = DWTForward(J=1, mode='zero', wave='db1')
        self.idwt = DWTInverse(mode="zero", wave="db1")

    def forward(self, pred, gt):
        # l1/l2 loss
        loss = {"l1": 0, "l2": 0}
        for _pred in pred["freq"]:
            _pred_rgb = self.dwt_to_img(_pred)
            _gt_rgb = F.interpolate(gt["img"], size=_pred_rgb.size(-1), mode='bilinear', align_corners=True)
            _gt_freq = self.img_to_dwt(_gt_rgb)
            loss["l1"] += self.l1_loss(_pred_rgb, _gt_rgb)
            loss["l2"] += self.l2_loss(_pred_rgb, _gt_rgb)
            loss["l1"] += self.l1_loss(_pred, _gt_freq)
            loss["l2"] += self.l2_loss(_pred, _gt_freq)
            
        # total loss
        loss["loss"] = 0
        for k, w in self.loss_weights.items():
            if loss[k] is not None:
                loss["loss"] += w * loss[k]
            else:
                del loss[k]
        return loss["loss"]

    def img_to_dwt(self, img):
        low, high = self.dwt(img)
        b, _, _, h, w = high[0].size()
        high = high[0].view(b, -1, h, w)
        freq = torch.cat([low, high], dim=1)
        return freq

    def dwt_to_img(self, img):
        b, c, h, w = img.size()
        low = img[:, :3, :, :]
        high = img[:, 3:, :, :].view(b, 3, 3, h, w)
        return self.idwt((low, [high]))

class ComparisonLoss(nn.Module): #work in progress,  but binary cross entropy loss for now
    def __init__(self):
        super().__init__()
        self.bcel = nn.BCELoss()
    
    def forward(self, pred, gt):
        return = self.bcel(pred, gt)

In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distibuted.parallel_loader as pl

# training loop 
def train_SSL(
        encoder, decoder, comparison, 
        recon_loss, comp_loss, loss_crit,
        train_loader: Iterable, optimizer: torch.optim.Optimizer,
        device: torch.device, writer, save_loc
        epoch: int, max_norm: float = 0, mixup_fn: Optional[Mixup] = None
    ):
    model.train(True)
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 50

    loader = pl.MpDeviceLoader(train_loader, device)

    #training 
    for i, train_data in enumerate(loader):
        imgs1_aug = distortImages(train_data["img1"]) # Apply distortion
        imgs2_aug = distortImages(train_data["img2"]) 

        
        optimizer.zero_grad()

        with torch.autocast():
            img1_enc = encoder(img1_aug) 
            img1_recon = decoder(img1_enc)

            img2_enc2 = encoder(img2_aug) 
            img2_recon = decoder(img2_enc)

            imgs_comp = comparison(torch.cat([img1_enc, img2_enc], dim=0))

            imgs_recon = torch.cat([imgs1_recon, imgs2_recon], dim=0) 
            imgs = torch.cat([imgs1, imgs2], dim=0) 

            loss = recon_loss(imgs_recon, imgs) + comp_loss(imgs_comp1, train_data["comp"])
            loss.backward()
        
            xm.optimizer_step(optimizer)

        writer.add_scalar("training loss", loss, i * imgs.size()[0])

        if loss < loss_crit :
            torch.save(
                {
                    "enc" : encoder.state_dict()
                    "dec" : decoder.state_dict(),
                    "comp" : comparison.state_dict()
                    "loss" : loss.state_dict()
                },
                save_loc
            )
            
            break 


    #evaluating
    #commented out because its not very useful
    '''loader = pl.MpDeviceLoader(test_loader, device)

    for i, img1, img2, same in enumerate(loader):
        imgs_aug = distortImages(imgs1) # Apply distortion

        with torch.autocast():
            imgs_recon = model(imgs_aug) 

            loss.append(loss(imgs_recon, imgs))

        writer.add_scalar("testing loss", loss, i * imgs.size()[0])'''

In [ ]:
# Summary: A combination of ResNeXt and Squeeze and Excitation Network (SENet) to 
# ResNeXt ref: https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py
# SENet ref: https://github.com/moskomule/senet.pytorch 

def conv1x1(in_channels, out_channels, stride = 1):
    """ 1x1 convolution"""
    return nn.Conv2d(
        in_channels, out_channels,
        kernel_size = 1, 
        stride = stride,
        bias = False
    )

def conv3x3(in_channels, out_channels, stride = 1, groups = 1, dilation = 1):
    """ 3x3 convolution """
    return nn.Conv2d(
        in_channels, out_channels,
        kernel_size = 3,
        stride = stride,
        padding = dilation,
        groups = groups,
        bias = False,
        dilation = dilation 
    )

def conv5x5(in_channels, out_channels, stride = 1, groups = 1, dilation = 1):
    return nn.Conv2d(
        in_channels, out_channels,
        kernel_size = 5,
        stride = stride,
        padding = dilation,
        groups = groups,
        bias = False,
        dilation = dilation 
    )

class BottleNeck(nn.Module):
    """ BottleNeck Layer in ResNet """

    expansion : int = 4 

    def __init__(
        self, in_channels, out_channels, 
        reduction=2, stride=1, downsample=None, num_groups=64,
        conv=conv3x3
    ):
        # if we want to use the expansion factor, we can just modify "out_channels"
        # out_channels = in_channels * self.expansion

        super().__init__()

        width = int(in_channels/reduction)

        # inplace is used for ReLU to reduce memory usage
        self.resnext_block = nn.Sequential(
            conv1x1(in_channels, width),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True),

            conv(width, width, stride=stride, groups=num_groups),
            nn.BatchNorm2d(width),
            nn.ReLU(inplace=True),

            conv1x1(width, out_channels),
            norm_layer(out_channels)
        )

        if downsample is not None:
            self.downsample = downsample

        self.activation = nn.ReLU(inplace=True)
    
    def forward(self, x):
        residual = x 
        out = self.resnext_block(x)

        if self.downsample is not None:
            residual = self.downsample(x)
            
        out += residual
        out = self.activation(out)
        return out

class SELayer(nn.Module):
    """ building block described in the SENet Paper and github"""
    def __init__(self, channel, reduction=16):
        super().__init__())
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, in_channel//reduction, bias=False),
            nn.ReLU(inplace=True)
            nn.Linear(in_channel//reduction, channel, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        batch, channel, _, _ = x.size()
        y = self.avg_pool(x).view(batch, channel)
        y = self.fc(y).view(batch, channel, 1, 1)
        return x * y.expand_as(x)

class ResNeXtLayer(nn.Module):
    """ basic layer described in the ResNeXt Paper """
    def __init__(self, in_channels, out_channels, num_blocks, num_groups=64, dilation=1, stride=1):
        super().__init__()
        downsample = nn.Sequential(
            conv1x1(in_channels, out_channels, stride),
            nn.BatchNorm2d(out_channels)
        )

        layers = []
        layers.append(BottleNeck(in_channels, out_channels, downsample=downsample, stride=stride))
        
        for _ in range(1, num_blocks):
            layers += [
                BottleNeck(out_channels, out_channels, dilation=dilation, num_groups=num_groups),
                SELayer(out_channels)
            ]
        
        self.layers = nn.Sequential(*layers)

        
    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
# CAPE: Continuous Augmented Positional Embeddings
# paper @ https://arxiv.org/2106.03143

class CAPE(nn.Module): # I have offically given up on encoding in general
    def __init__(
        self, model_dim 
        max_global_shift = 0.0, max_local_shift = 0.0, max_global_scaling = 1.0, 
    ):
        self.max_global_shift = max_global_shift
        self.max_local_shift = max_local_shift
        self.max_global_scaling = max_global_scaling

        self.register_buffer('content_scale', nn.Tensor([math.sqrt(model_dim)]))

    def forward(self, patches):
        return (patches * self.content_scale) + self.compute_pos_emb(patches)
    
    def compute_pos_emb(self, patches):
        batch, height, width, channel = patches.shape()

        x = torch.zeros


In [ ]:
# TransCNN + Universal Transformer 
# Idea : each layer repeated via ACT, then down sampled
# Idea : direct downsampling for residual

class CNNAttention(nn.Module):
    def __init__(self, total_dim, head_dim, conv=conv1x1, grid_size=1, downsample_rate=1, drop=0):
        super().__init__()
        self.num_heads = total_dim // head_dim # area of previous step / area of head
        self.head_dim = head_dim 
        self.side_len = self.head_dim ** -0.5
        self.grid_size = grid_size

        self.norm = nn.BatchNorm2d()
        self.qkv = conv(total_dim, total_dim * 3)
        self.proj = nn.Conv2d(total_dim, total_dim)
        self.drop = nn.Dropout2d(drop, inplace=True)

        if self.grid_size > 1:
            self.q = conv(total_dim, total_dim)
            self.kv = conv(total_dim, total_dim * 2)

            self.grid_norm = nn.BatchNorm2d(total_dim)
            self.avg_pool = nn.AvgPool2d(total_dim)
            self.downsample_norm = nn.BatchNorm2d(total_dim)

    def forward(self, x):
        batch, channels, height, width = x.shape()
        qkv = self.qkv(self.norm(x))

        if self.grid_size > 1:
            # compute grid based/local attention
            grid_h, grid_w = height // self.grid_size, width // self.grid_size # H/G, W/G
            qkv = qkv.reshape(
                batch, 3, # q, k, v
                self.num_heads, self.head_dim, 
                grid_h, self.grid_size,
                grid_w, self.grid_size
            ) # ref. the dimensions of this space is R^ Batch * QKV * Head * Size * H/G * G * W/G * G
            qkv = qkv.permute(1, 0, 2, 4, 6, 5, 7, 3) # R^ QKV * Batch * Head * H/G * W/G * G * G * Size 
            qkv = qkv.reshape(3, -1, self.grid_size ** 2, self.head_dim) # R^ QKV * (Batch * Head * H/ * G * G * Size 
            q, k, v = qkv[0], qkv[1], qkv[2]

            attn = (q @ k.transpose(-2, -1)) * self.side_len # transpose k -> R^ Batch * G * G * (W * H)
            attn = attn.softmax(dim=-1)
            grid_x = (attn @ v).reshape(
                batch, self.num_heads, 
                grid_h, grid_w, 
                self.grid_size, self.grid_size, 
                self.head_dim 
            ) # R^ Batch * Head * H/G * W/G * G * G * Size, same as after permute
            grid_x = self.grid_norm(x + grid_x) #residue and normalisation

            # transform qkv for computing global attention
            q = self.q(grid_x).reshape(batch, self.num_heads, self.head_dim, -1) # R^ Batch * Head * Size * (H * W)
            q = q.transpose(-2, -1) # R^ Batch * Head * (H * W) * Size 
            kv = self.kv(self.downsample_norm(self.avg_pool(grid_x)))
            kv = kv.reshape(batch, 2, self.num_heads, self.head_dim, -1) # R^ Batch * KV * Head * Size * (H * W)
            kv = kv.permute(1, 0, 2, 4, 3) # R^ KV * Batch * Head *  (H * W) * Size 
            k, v = kv[0], kv[1] # R^ Batch * Head * (H * W) * Size
        else: 
            # transform qkv for computing global attention
            qkv = qkv.reshape(B, 3, self.num_heads, self.head_dim, -1) # R^ Batch * QKV * Head * Size * (H * W)
            qkv = qkv.permute(1, 0, 2, 4, 3) # R^ QKV * Batch * Head * (H * W) * Size
            q, k, v = qkv[0], qkv[1], qkv[2]
        
        # compute global attention
        attn = (q @ k.transpose(-2, -1)) * self.side_len 
        attn = attn.softmax(dim=1)
        global_x = (attn @ v).transpose(-2, -1).reshape(batch, channel, height, width)

        # residue
        if self.grid_size > 1
            global_x += grid_x
        x = self.drop(self.proj(global_x))

        return x 

class ACT(nn.Module):

    threshold = 1 - 0.1

    def __init__(self, module, size, max_steps, activation=nn.Sigmoid):
        super().__init__()
        self.module = module
        self.activation = activation()
        self.fc = nn.Linear(size, 1) # What if we replace linear with conv2d?

        self.max_steps = max_steps
        # !!!be sure to initialise self.p!!!
    
    def forward(self, state):
        # change the shape of input to 3d for ACT
        shape = state.size()
        if len(shape) > 3
            state = state.view((*shape[:1], )) 
        batch, size, _ = state.size()

        halting_probablity = torch.zeros(batch, size)
        remainders = torch.zeros(batch, size)
        n_updates = torch.zeros(batch, size)
        previous_state = torch.zeros_like(inputs)

        def should_continue(h, n, m):
            return (h < self.threshold and n < m).byte().any()
        while should_continue(halting_probability, n_updates, self.max_steps):
            # we are avoiding timing signals because we have our own RPE 
            state = self.activation(self.fc(state)).squeeze(-1) 

            # calculate masks for which ones to halt
            still_running = (halting_probability < 1.0).float()
            new_halted = (halting_probability + state * still_running > self.threshold).float() * still_running
            still_running = (halting_probability + state * still_running <= self.threshold).float() * still_running

            # halt parameters and increment remainders
            halting_probability += state * still_running
            remainders += new_halted * (1 - halting_probability)
            halting_probability += new_halted * remainders 
            n_updates += still_running + new_halted
            # compute weights to apply to the state and output
            update_weights = state * still_running + new_halted * remainders 
            
            state = self.module(state)
            previous_state = state * update_weights.unsqueeze(-1) + previous_state * (1 - update_weights.unsqueeze(-1))

            step += 1
            
        # change the shape back to the original
        previous_state = previous_state.view((*shape, ))
        return previous_state, (remainders, n_updates)

class TransDownsample(nn.Module):
    def __init__(self, in_dim, out_dim, activation=nn.SiLU):
        super().__init__()
        self.conv = conv3x3(in_dim, out_dim, stride=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.residual = nn.Conv2d(in_dim, out_dim, 1)
        self.norm1 = nn.BatchNorm2d(out_dim)
        self.norm2 = nn.BatchNorm2d(out_dim)
        self.act = activation(inplace=True)

    def forward(self, x):
        x1 = self.norm1(self.conv(x))
        x2 = self.norm2(self.residual(self.pool(x)))
        x = self.act(x1 + x2)
        return x

class TransCNNLayer(nn.Module):
    def __init__(
        self, in_dim, out_dim, depth, max_steps, 
        head_dim=64, conv=conv1x1, grid_size=1, downsample_rate=1, drop=0,
        bnconv=conv3x3
    ):
        super().__init__()

        self.down_sample = TransDownsample(in_dim, out_dim)

        trans_cnn = []
        for i in range(depth):
            transcnn_block = nn.Sequential(
                CNNAttention(
                    out_dim,
                    head_dim,
                    conv=conv,
                    grid_size=grid_size,
                    downsample_rate=downsample_rate,
                    drop=drop
                ),
                BottleNeck(total_dim, total_dim, conv=bnconv)
            )
            trans_cnn.append(ACT(transcnn_block, total_dim, max_steps))
        self.trans_cnn = nn.ModuleList(*seq)
        
    
    def forward(self, x):
        for trans_cnn in self.trans_cnn:
            x, _ = trans_cnn(x)
        return x

In [ ]:
# MobileStyleGAN w/o the style
# Q: Can we just ctrl c+p this?
# A: Yes! Yes we can!

from pytorch_wavelets.dwt.lowlevel import *

def _SFB2D(low, highs, g0_row, g1_row, g0_col, g1_col, mode):
    mode = int_to_mode(mode)

    lh, hl, hh = torch.unbind(highs, dim=2)
    lo = sfb1d(low, lh, g0_col, g1_col, mode=mode, dim=2)
    hi = sfb1d(hl, hh, g0_col, g1_col, mode=mode, dim=2)
    y = sfb1d(lo, hi, g0_row, g1_row, mode=mode, dim=3)

    return y

class DWTInverse(nn.Module):
    """ Performs a 2d DWT Inverse reconstruction of an image
    Args:
        wave (str or pywt.Wavelet): Which wavelet to use
        C: deprecated, will be removed in future
    """
    def __init__(self, wave='db1', mode='zero', trace_model=False):
        super().__init__()
        if isinstance(wave, str):
            wave = pywt.Wavelet(wave)
        if isinstance(wave, pywt.Wavelet):
            g0_col, g1_col = wave.rec_lo, wave.rec_hi
            g0_row, g1_row = g0_col, g1_col
        else:
            if len(wave) == 2:
                g0_col, g1_col = wave[0], wave[1]
                g0_row, g1_row = g0_col, g1_col
            elif len(wave) == 4:
                g0_col, g1_col = wave[0], wave[1]
                g0_row, g1_row = wave[2], wave[3]
        # Prepare the filters
        filts = prep_filt_sfb2d(g0_col, g1_col, g0_row, g1_row)
        self.register_buffer('g0_col', filts[0])
        self.register_buffer('g1_col', filts[1])
        self.register_buffer('g0_row', filts[2])
        self.register_buffer('g1_row', filts[3])
        self.mode = mode
        self.trace_model = trace_model

    def forward(self, coeffs):
        yl, yh = coeffs
        ll = yl
        mode = mode_to_int(self.mode)

        for h in yh[::-1]:
            if h is None:
                h = torch.zeros(ll.shape[0], ll.shape[1], 3, ll.shape[-2],
                                ll.shape[-1], device=ll.device)

            if ll.shape[-2] > h.shape[-2]:
                ll = ll[...,:-1,:]
            if ll.shape[-1] > h.shape[-1]:
                ll = ll[...,:-1]
            if not self.trace_model:
                ll = SFB2D.apply(ll, h, self.g0_col, self.g1_col, self.g0_row, self.g1_row, mode)
            else:
                ll = _SFB2D(ll, h, self.g0_col, self.g1_col, self.g0_row, self.g1_row, mode)
        return ll

class IDWTUpsample(nn.Module):
    def __init__(
            self,
            channels_in,
    ):
        super().__init__()
        self.channels = channels_in // 4
        assert self.channels * 4 == channels_in
        # upsample
        self.idwt = DWTInverse(mode='zero', wave='db1')

    def forward(self, x):
        b, _, h, w = x.size()
        low = x[:, :self.channels]
        high = x[:, self.channels:]
        high = high.view(b, self.channels, 3, h, w)
        x = self.idwt((low, [high]))
        return x

class ModulatedConv2d(nn.Module):
    def __init__(
            self,
            channels_in,
            channels_out,
            kernel_size,
    ):
        super().__init__()
        # create conv
        self.weight = nn.Parameter(
            torch.randn(channels_out, channels_in, kernel_size, kernel_size)
        )
        # some service staff
        self.scale = 1.0 / math.sqrt(channels_in * kernel_size ** 2)
        self.padding = kernel_size // 2

    def forward(self, x):
        x = F.conv2d(x, self.weight, padding=self.padding)
        return x


class ModulatedDWConv2d(nn.Module):
    def __init__(
            self,
            channels_in,
            channels_out,
            kernel_size,
    ):
        super().__init__()
        # create conv
        self.weight_dw = nn.Parameter(
            torch.randn(channels_in, 1, kernel_size, kernel_size)
        )
        self.weight_permute = nn.Parameter(
            torch.randn(channels_out, channels_in, 1, 1)
        )
        # some service staff
        self.scale = 1.0 / math.sqrt(channels_in * kernel_size ** 2)
        self.padding = kernel_size // 2

    def forward(self, x):
        x = F.conv2d(x, self.weight_dw, padding=self.padding, groups=x.size(1))
        x = F.conv2d(x, self.weight_permute)
        return x

class StyledConv2d(nn.Module):
    def __init__(
        self,
        channels_in,
        channels_out,
        kernel_size,
        conv_module
    ):
        super().__init__()

        self.conv = conv_module(
            channels_in,
            channels_out,
            kernel_size,
        )

        self.bias = nn.Parameter(torch.zeros(1, channels_out, 1, 1))
        self.act = nn.LeakyReLU(0.2)

    def forward(self, input):
        out = self.conv(input)
        out = self.act(out + self.bias)
        return out

class MultichannelImage(nn.Module):
    def __init__(
            self,
            channels_in,
            channels_out,
            kernel_size=1
    ):
        super().__init__()
        self.conv = ModulatedConv2d(channels_in, channels_out, kernel_size, demodulate=False)
        self.bias = nn.Parameter(torch.zeros(1, channels_out, 1, 1))

    def forward(self, hidden):
        out = self.conv(hidden)
        out = out + self.bias
        return out

class MobileSynthesisBlock(nn.Module):
    def __init__(
            self,
            channels_in,
            channels_out,
            kernel_size=3,
            conv_module
    ):
        super().__init__()
        self.up = IDWTUpsample(channels_in)
        self.conv1 = StyledConv2d(
            channels_in // 4,
            channels_out,
            kernel_size,
            conv_module=conv_module
        )
        self.conv2 = StyledConv2d(
            channels_out,
            channels_out,
            kernel_size,
            conv_module=conv_module
        )
        self.to_img = MultichannelImage(
            channels_in=channels_out,
            channels_out=12,
            kernel_size=1
        )

    def forward(self, hidden):
        hidden = self.up(hidden)
        hidden = self.conv1(hidden)
        hidden = self.conv2(hidden)
        img = self.to_img(hidden)
        return hidden, img

    def wsize(self):
        return 3

In [ ]:
class Encoder(nn.Module):
    def __init__(self, height, width, head_dim=64, num_channels=3):
        # conv7x7 + ResNet [1-3] + TransCNN [4-5]
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(num_channels, 64, 7, stride=2),
            nn.MaxPool2d(3, stride=2),
            
            ResNextLayer(64, 256, 3, num_groups=32, stride=2),
            ResNextLayer(256, 512, 4, num_groups=32, stride=2),
        
            TransCNNLayer(512, 256, 4, 8, bnconv=conv5x5, head_dim=head_dim),
            TransCNNLayer(256, 512, 2, 4, bnconv=conv3x3, head_dim=head_dim)
        )
    
    def forward(self, x):
        x = self.seq(x)
        return x

class Decoder(nn.Module):
    def __init__(
            self, head_dim=64, conv=conv1x1, grid_size=1,
            trans_channels = [512, 512, 512],
            cnn_channels = [512, 512, 256, 128, 64]
    ):
        super().__init__()

        # We need to add 
        # Backwards version of TransCNN
        self.trans_layers = nn.ModuleList()
        for i, channels_out in enumerate(trans_channels):
            transcnn_block = CNNAttention(
                channels_out,
                head_dim,
                conv=conv,
                grid_size=grid_size,
                downsample_rate=1,
                drop=0
            )
            trans_layers.append(ACT(transcnn_block, total_dim, max_steps))

        self.cnn_layers = nn.ModuleList()
        channels_in = cnn_channels[0]
        for i, channels_out in enumerate(cnn_channels[1:]):
            self.cnn_layers.append(
                MobileSynthesisBlock(
                    channels_in,
                    channels_out,
                    3,
                    conv_module=ModulatedDWConv2d
                )
            )
            channels_in = channels_out

        self.idwt = DWTInverse(mode="zero", wave="db1")
        self.register_buffer("device_info", torch.zeros(1))
        self.trace_model = False

    def forward(self, x):
        out = {"freq": [], "img": None}

        for trans_cnn in self.trans_layers:
            x, _ = trans_cnn(x)
        
        for i, m in enumerate(self.cnn_layers):
            x, freq = m(x)
            out["freq"].append(freq)

        out["img"] = self.dwt_to_img(out["freq"][-1])
        return out

    def dwt_to_img(self, img):
        b, c, h, w = img.size()
        low = img[:, :3, :, :]
        high = img[:, 3:, :, :].view(b, 3, 3, h, w)
        return self.idwt((low, [high]))

    def wsize(self):
        return len(self.cnn_layers) * self.cnn_layers[0].wsize() + 2

class CompHead(nn.Module):
    def __init__(self, in_size, comp_size):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(in_size, in_size),
            nn.Linear(in_size, comp_size)
        )
    
    def forward(self, x):
        x = x.flatten()
        x = self.seq(x)

        return x